**Import the Dependencies** 💦💙

In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as img 
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import keras



**Loading Data** 💦💙

In [ ]:
with_mask = os.listdir('data/with_mask')
print(with_mask[0:5])


In [ ]:
without_mask = os.listdir('data/without_mask')
print(without_mask[0:5])


In [ ]:
print (len(with_mask))
print (len(without_mask))


**Creating Labels** 💦💙
- with mask >> 1 
- without mask >> 0  

In [5]:
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828


In [ ]:
print(with_mask_labels[0:5])
print(without_mask_labels[0:5])


In [ ]:
# combine 2 sets 

labels = with_mask_labels + without_mask_labels 
print(len(labels))


**Displaying Images** 💦💙


In [ ]:
image = img.imread('data/with_mask/with_mask_1.jpg')
img_plot = plt.imshow(image)
plt.show()



In [ ]:
image = img.imread('data/without_mask/without_mask_1000.jpg')
img_plot = plt.imshow(image)
plt.show()


**Image Processing**💦💙
    

1. Resize the Images
2. Convert the images to numpy arrays 

In [ ]:
# convert images to numpy arrays+

with_mask_path = 'data/with_mask/'

data = []

for img_file in with_mask:

  image = Image.open(with_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)



without_mask_path = 'data/without_mask/'


for img_file in without_mask:

  image = Image.open(without_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)



In [ ]:
type(data)


In [ ]:
len(data)


In [ ]:
data[0]


In [ ]:
data[0].shape


In [15]:
# converting image list and label list to numpy arrays


X = np.array(data)
Y = np.array(labels)


In [ ]:
print(X.shape)
print(Y.shape)


In [ ]:
print(Y)


**Split Data to train and test** 💦💙

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)


**Scaling The Data** 💦💙

In [19]:
X_train_scaled = X_train/255

X_test_scaled = X_test/255


In [ ]:
X_train_scaled[0]


**Building a Convolutional Neural Networks (CNN)** 💦💙


In [ ]:
num_of_classes = 2  # binary classification , The network will output two probabilities representing each class.

model = keras.Sequential() # This model is a linear stack of layers where each layer has one input tensor and one output tensor.

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
# Conv2D(32): This adds a 2D convolutional layer with 32 filters (also known as kernels).
# kernel_size=(3,3): The size of each filter is 3x3 pixels.
# activation='relu': The Rectified Linear Unit (ReLU) activation function is applied, which introduces non-linearity by setting negative values to 0 and keeping positive values as they are.
# input_shape=(128,128,3): This specifies the shape of the input data, which in this case is a 128x128 image with 3 channels (likely RGB).
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
# MaxPooling2D(pool_size=(2,2)): This layer downsamples the input by taking the maximum value from each 2x2 block of pixels, 
# effectively reducing the spatial dimensions (width and height) by half. 
# This helps reduce computational cost and also prevents overfitting by making the model more invariant to small shifts in the input.


model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu')) #Second Convolutional Layer

model.add(keras.layers.MaxPooling2D(pool_size=(2,2))) #Second Max Pooling Layer

model.add(keras.layers.Flatten())
# Flatten(): This layer flattens the multi-dimensional output from the previous convolutional and pooling layers into a 1D vector.
# This is necessary before feeding the data into a fully connected layer (Dense layer).

model.add(keras.layers.Dense(128, activation='relu'))
# Dense(128): This adds a fully connected (dense) layer with 128 neurons.
model.add(keras.layers.Dropout(0.5))
# Dropout(0.5): Dropout is a regularization technique where 50% of the neurons in this layer are randomly ignored (dropped) during training to prevent overfitting. 
# This helps make the model more robust by ensuring it doesn't rely too heavily on any one neuron.
model.add(keras.layers.Dense(64, activation='relu'))  #Second Fully Connected (Dense) Layer
model.add(keras.layers.Dropout(0.5)) #Second Dropout Layer


model.add(keras.layers.Dense(num_of_classes, activation='sigmoid')) #Output Layer


In [22]:
# compile the neural network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])


In [ ]:
# training the neural network
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=5)


**Model Evaluation** 💦💙

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)


In [ ]:
h = history

# plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()


**Predictive System** 💦💙

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

# Check if the image was loaded successfully
if input_image is None:
    print("Error: Image not found or unable to load.")
else:
    # Display the image in a window named 'Input Image'
    cv2.imshow('Input Image', input_image)

    # Wait until a key is pressed
    cv2.waitKey(0)
    
    # Destroy all OpenCV windows
    cv2.destroyAllWindows()

    # Resize the image to the input size expected by the model
    input_image_resized = cv2.resize(input_image, (128, 128))

    # Normalize the image
    input_image_scaled = input_image_resized / 255.0

    # Reshape the image to fit the model input
    input_image_reshaped = np.reshape(input_image_scaled, [1, 128, 128, 3])

    # Predict using the model
    input_prediction = model.predict(input_image_reshaped)

    # Print the prediction probabilities
    print("Prediction probabilities:", input_prediction)

    # Get the predicted class
    input_pred_label = np.argmax(input_prediction)

    # Print the predicted label
    print("Predicted label:", input_pred_label)

    # Interpret the result
    if input_pred_label == 1:
        print('The person in the image is wearing a mask')
    else:
        print('The person in the image is not wearing a mask')
